In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
premise_file = "../Resources/LicensePremise.csv"
premise_df = pd.read_csv(premise_file)
premise_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Resources/LicensePremise.csv'

In [ ]:
county_file = "../Resources/CountyLicenseCount.csv"
county_df = pd.read_csv(county_file)
county_df.head()

### Transform premise DataFrame

In [4]:
# Create a filtered dataframe from specific columns
premise_cols = ["License Serial Number", "Premises Name", "County ID Code"]
premise_transformed= premise_df[premise_cols].copy()

# Rename the column headers
premise_transformed = premise_transformed.rename(columns={"License Serial Number": "id",
                                                          "Premises Name": "premise_name",
                                                          "County ID Code": "county_id"})

# Clean the data by dropping duplicates and setting the index
premise_transformed.drop_duplicates("id", inplace=True)
premise_transformed.set_index("id", inplace=True)

premise_transformed.head()

,premise_name,county_id
id,,
1311660,CANA ARRIBA GROCERY NO 2 INC,0
1310023,JOHANA GROCERY & DELI CORP,0
1310024,TKO BEVERAGES LLC,1
1311663,181 LEXINGTON AVENUE BBQ LLC,2
1310029,AZIZ DELI & GRILL CORP,1


### Transform county DataFrame

In [5]:
county_cols = ["ID", "County Name (Licensee)", "County ID Code", "License Count"]
county_transformed = county_df[county_cols].copy()

# Rename the column headers
county_transformed = county_transformed.rename(columns={"ID": "id",
                                                         "County Name (Licensee)": "county_name",
                                                         "License Count": "license_count",
                                                         "County ID Code": "county_id"})

# Set index
county_transformed.set_index("id", inplace=True)

county_transformed.head()

,county_name,county_id,license_count
id,,,
0,ALBANY,5,77
1,ALLEGANY,59,4
2,BRONX,0,104
3,BROOME,35,14
4,CATTARAUGUS,41,9


### Create database connection

In [6]:
connection_string = "postgres:postgres@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [7]:
# Confirm tables
engine.table_names()

['county', 'premise']

### Load DataFrames into database

In [8]:
premise_transformed.to_sql(name='premise', con=engine, if_exists='append', index=True)

In [9]:
county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)